In [2]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter
from scipy import stats


In [5]:
stocks = pd.read_csv('sp_500_stocks.csv')


In [9]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token=sk_f35e72cbeb9d4cc09281640e5f1f899a'
data= requests.get(api_url).json()
data

{'avgTotalVolume': 54011435,
 'calculationPrice': 'close',
 'change': -1.08,
 'changePercent': -0.00555,
 'close': None,
 'closeSource': 'official',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': 'Close',
 'highTime': 1703278800223,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 193.59,
 'iexCloseTime': 1703278799420,
 'iexLastUpdated': 1703278799420,
 'iexMarketPercent': 0.021553008554338584,
 'iexOpen': 195.165,
 'iexOpenTime': 1703255400093,
 'iexRealtimePrice': 193.59,
 'iexRealtimeSize': 249,
 'iexVolume': 800685,
 'lastTradeTime': 1703278799743,
 'latestPrice': 193.6,
 'latestSource': 'Close',
 'latestTime': 'December 22, 2023',
 'latestUpdate': 1703278800223,
 'latestVolume': None,
 'low': None,
 'lowSource': 'Close',
 'lowTime': 17

In [12]:
price = data['latestPrice']
pe_ratio  = data['peRatio']


In [61]:
def chunks(lst, n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings=[]
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
my_columns=['Ticker','Price','Price-to-Earnings Ratio','Number of Shares to Buy']
my_columns

['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [62]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/v1/stock/market/batch?symbols={symbol_string}&types=quote&token=sk_f35e72cbeb9d4cc09281640e5f1f899a'
    data= requests.get(batch_api_call_url).json()
    temp_data = []
    for symbol in symbol_string.split(','):
            temp_data.append([
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'       
            ])
    final_dataframe = pd.concat([final_dataframe, pd.DataFrame(temp_data, columns=my_columns)], ignore_index=True)
final_dataframe


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,139.57,33.31,N/A
1,AAL,14.31,5.34,N/A
2,AAP,61.25,19.63,N/A
3,AAPL,193.60,31.58,N/A
4,ABBV,154.94,42.45,N/A
...,...,...,...,...
491,YUM,130.00,24.71,N/A
492,ZBH,120.56,53.35,N/A
493,ZBRA,269.41,30.03,N/A
494,ZION,44.00,4.70,N/A


In [63]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending=True, inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio']>0]
final_dataframe = final_dataframe[:50]


In [64]:
#final_dataframe.drop('index',axis=1, inplace=True)

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
69,BRK.B,356.47,0.01,N/A
188,FRC,3.51,0.43,N/A
134,DISH,5.00,2.51,N/A
156,EMR,96.44,4.21,N/A
460,VLO,132.20,4.50,N/A
494,ZION,44.00,4.70,N/A
447,UAL,42.55,4.94,N/A
199,GM,36.02,5.07,N/A
22,ALB,150.09,5.32,N/A
1,AAL,14.31,5.34,N/A


In [65]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of the portfolio ')
    
    try:
        val = float(portfolio_size)
    except ValueError:
        print('This is not a number')
        portfolio_size = input('Enter the size of the portfolio ')
portfolio_input()

Enter the size of the portfolio 1000000


In [69]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row,'Price'])


In [71]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BRK.B,356.47,0.01,56
1,FRC,3.51,0.43,5698
2,DISH,5.00,2.51,4000
3,EMR,96.44,4.21,207
4,VLO,132.20,4.50,151
5,ZION,44.00,4.70,454
6,UAL,42.55,4.94,470
7,GM,36.02,5.07,555
8,ALB,150.09,5.32,133
9,AAL,14.31,5.34,1397


In [82]:
#Build a better Strategy based on different mutiples.
symbol = 'AAPL'
batch_api_call_url = f'https://cloud.iexapis.com/v1/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token=sk_f35e72cbeb9d4cc09281640e5f1f899a'
data = requests.get(batch_api_call_url).json()
#print(data['AAPL']['advanced-stats'])
#P/E ratio
pe_ratio = data[symbol]['quote']['peRatio']

#P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratip
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

#EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value / ebitda

#EV/Gross_Profit
gross_profit =data[symbol]['advanced-stats']['grossProfit']

ev_to_gross_profit = enterprise_value/gross_profit



In [83]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Share to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile','Price-to-Book Ratio',
    'PB Percentile','Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA Ratio',
    'EV/EBITDA Percentile',
    'EV/Gross Profit Ratio',
    'EV/Gross Profit Percentile',
    'RV Score'
]
rv_dataframe=pd.DataFrame(columns = rv_columns)
rv_dataframe

,Ticker,Price,Number of Share to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/Gross Profit Ratio,EV/Gross Profit Percentile,RV Score


In [96]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/v1/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token=sk_f35e72cbeb9d4cc09281640e5f1f899a'
    data= requests.get(batch_api_call_url).json()
    temp2_data=[]
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit =data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_ebitda = enterprise_value / ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        temp2_data.append([
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                     data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A'
                              
            ])
    rv_dataframe = pd.concat([rv_dataframe, pd.DataFrame(temp2_data, columns=rv_columns)], ignore_index=True)
rv_dataframe

        
        
   

,Ticker,Price,Number of Share to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/Gross Profit Ratio,EV/Gross Profit Percentile,RV Score
0,A,139.57,N/A,33.31,N/A,7.00,N/A,5.9800,N/A,25.934351,N/A,12.132636,N/A,N/A
1,AAL,14.31,N/A,5.34,N/A,-1.82,N/A,0.1767,N/A,4.834282,N/A,0.609653,N/A,N/A
2,AAP,61.25,N/A,19.63,N/A,1.38,N/A,0.3226,N/A,8.296798,N/A,1.088871,N/A,N/A
3,AAPL,193.60,N/A,31.58,N/A,48.45,N/A,7.8600,N/A,24.324796,N/A,18.093893,N/A,N/A
4,ABBV,154.94,N/A,42.45,N/A,22.62,N/A,4.9600,N/A,12.768321,N/A,8.853694,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,130.00,N/A,24.71,N/A,-4.45,N/A,5.1600,N/A,19.695699,N/A,8.936356,N/A,N/A
492,ZBH,120.56,N/A,53.35,N/A,2.01,N/A,3.4600,N/A,15.850294,N/A,5.827471,N/A,N/A
493,ZBRA,269.41,N/A,30.03,N/A,4.59,N/A,2.7200,N/A,18.404577,N/A,6.800335,N/A,N/A
494,ZION,44.00,N/A,4.70,N/A,1.34,N/A,1.3100,N/A,3.589508,N/A,1.354644,N/A,N/A


In [101]:
for column in ['Price-to-Earnings Ratio','Price-to-Book Ratio','Price-to-Sales Ratio','EV/EBITDA Ratio', 'EV/Gross Profit Ratio']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [102]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Share to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/Gross Profit Ratio,EV/Gross Profit Percentile,RV Score


In [107]:
metrics= {
    'Price-to-Earnings Ratio':'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA Ratio':'EV/EBITDA Percentile',
    'EV/Gross Profit Ratio': 'EV/Gross Profit Percentile',
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
rv_dataframe

,Ticker,Price,Number of Share to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/Gross Profit Ratio,EV/Gross Profit Percentile,RV Score
0,A,139.57,N/A,33.31,0.747984,7.00,0.796371,5.9800,0.835685,25.934351,0.883065,12.132636,0.832661,N/A
1,AAL,14.31,N/A,5.34,0.100806,-1.82,0.102823,0.1767,0.012097,4.834282,0.054435,0.609653,0.018145,N/A
2,AAP,61.25,N/A,19.63,0.443548,1.38,0.21875,0.3226,0.038306,8.296798,0.195565,1.088871,0.056452,N/A
3,AAPL,193.60,N/A,31.58,0.711694,48.45,0.979839,7.8600,0.888105,24.324796,0.856855,18.093893,0.945565,N/A
4,ABBV,154.94,N/A,42.45,0.842742,22.62,0.959677,4.9600,0.766129,12.768321,0.421371,8.853694,0.66129,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,130.00,N/A,24.71,0.552419,-4.45,0.09879,5.1600,0.78629,19.695699,0.727823,8.936356,0.667339,N/A
492,ZBH,120.56,N/A,53.35,0.895161,2.01,0.362903,3.4600,0.59375,15.850294,0.556452,5.827471,0.407258,N/A
493,ZBRA,269.41,N/A,30.03,0.675403,4.59,0.673387,2.7200,0.530242,18.404577,0.679435,6.800335,0.485887,N/A
494,ZION,44.00,N/A,4.70,0.092742,1.34,0.208669,1.3100,0.235887,3.589508,0.028226,1.354644,0.08871,N/A


In [112]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentile = []
    for metric in metrics:
        value_percentile.append(rv_dataframe.loc[row, metrics[metric]])
        rv_dataframe.loc[row, 'RV Score']= mean(value_percentile)

rv_dataframe

,Ticker,Price,Number of Share to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/Gross Profit Ratio,EV/Gross Profit Percentile,RV Score
0,A,139.57,N/A,33.31,0.747984,7.00,0.796371,5.9800,0.835685,25.934351,0.883065,12.132636,0.832661,0.819153
1,AAL,14.31,N/A,5.34,0.100806,-1.82,0.102823,0.1767,0.012097,4.834282,0.054435,0.609653,0.018145,0.057661
2,AAP,61.25,N/A,19.63,0.443548,1.38,0.21875,0.3226,0.038306,8.296798,0.195565,1.088871,0.056452,0.190524
3,AAPL,193.60,N/A,31.58,0.711694,48.45,0.979839,7.8600,0.888105,24.324796,0.856855,18.093893,0.945565,0.876411
4,ABBV,154.94,N/A,42.45,0.842742,22.62,0.959677,4.9600,0.766129,12.768321,0.421371,8.853694,0.66129,0.730242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,130.00,N/A,24.71,0.552419,-4.45,0.09879,5.1600,0.78629,19.695699,0.727823,8.936356,0.667339,0.566532
492,ZBH,120.56,N/A,53.35,0.895161,2.01,0.362903,3.4600,0.59375,15.850294,0.556452,5.827471,0.407258,0.563105
493,ZBRA,269.41,N/A,30.03,0.675403,4.59,0.673387,2.7200,0.530242,18.404577,0.679435,6.800335,0.485887,0.608871
494,ZION,44.00,N/A,4.70,0.092742,1.34,0.208669,1.3100,0.235887,3.589508,0.028226,1.354644,0.08871,0.130847


In [118]:
rv_dataframe.sort_values('RV Score', ascending= True, inplace = True)
rv_dataframe=rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace = True)
rv_dataframe


,Ticker,Price,Number of Share to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/Gross Profit Ratio,EV/Gross Profit Percentile,RV Score
0,FRC,3.51,N/A,0.43,0.084677,0.0456,0.104839,0.0934,0.002016,0.270279,0.014113,0.094878,0.006048,0.042339
1,AAL,14.31,N/A,5.34,0.100806,-1.8200,0.102823,0.1767,0.012097,4.834282,0.054435,0.609653,0.018145,0.057661
2,GM,36.02,N/A,5.07,0.096774,0.6623,0.110887,0.2868,0.028226,1.335170,0.016129,0.943033,0.040323,0.058468
3,WBA,26.22,N/A,-7.34,0.058468,1.1300,0.164315,0.1629,0.008065,-6.746861,0.012097,1.144165,0.060484,0.060685
4,C,50.86,N/A,8.11,0.133065,0.5123,0.108871,0.6588,0.092742,4.290943,0.040323,0.693547,0.024194,0.079839
5,UNM,44.94,N/A,7.25,0.110887,0.9113,0.135081,0.7157,0.103831,4.229361,0.038306,0.715743,0.030242,0.083669
6,UAL,42.55,N/A,4.94,0.094758,1.5800,0.265121,0.2659,0.022177,3.300175,0.024194,0.510452,0.012097,0.083669
7,F,12.35,N/A,8.07,0.131048,1.1000,0.155242,0.2787,0.024194,2.508064,0.018145,1.471766,0.102823,0.08629
8,KSS,27.67,N/A,-21.28,0.034274,0.8167,0.118952,0.1747,0.010081,9.163331,0.225806,1.245997,0.068548,0.091532
9,SYF,37.94,N/A,7.03,0.108871,1.2000,0.178427,0.7345,0.106855,4.426202,0.044355,0.980498,0.044355,0.096573


In [119]:
portfolio_input()

Enter the size of the portfolio 1000000


In [125]:
position_size = float(portfolio_size)/len(rv_dataframe.index)

for row in rv_dataframe.index:
    rv_dataframe.loc[row,'Number of Share to Buy'] = math.floor(position_size/rv_dataframe.loc[row,'Price'])
rv_dataframe

,Ticker,Price,Number of Share to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA Ratio,EV/EBITDA Percentile,EV/Gross Profit Ratio,EV/Gross Profit Percentile,RV Score,Number of Shares to Buy
0,FRC,3.51,5698,0.43,0.084677,0.0456,0.104839,0.0934,0.002016,0.270279,0.014113,0.094878,0.006048,0.042339,5698.005698
1,AAL,14.31,1397,5.34,0.100806,-1.8200,0.102823,0.1767,0.012097,4.834282,0.054435,0.609653,0.018145,0.057661,1397.624039
2,GM,36.02,555,5.07,0.096774,0.6623,0.110887,0.2868,0.028226,1.335170,0.016129,0.943033,0.040323,0.058468,555.247085
3,WBA,26.22,762,-7.34,0.058468,1.1300,0.164315,0.1629,0.008065,-6.746861,0.012097,1.144165,0.060484,0.060685,762.776506
4,C,50.86,393,8.11,0.133065,0.5123,0.108871,0.6588,0.092742,4.290943,0.040323,0.693547,0.024194,0.079839,393.236335
5,UNM,44.94,445,7.25,0.110887,0.9113,0.135081,0.7157,0.103831,4.229361,0.038306,0.715743,0.030242,0.083669,445.037828
6,UAL,42.55,470,4.94,0.094758,1.5800,0.265121,0.2659,0.022177,3.300175,0.024194,0.510452,0.012097,0.083669,470.035253
7,F,12.35,1619,8.07,0.131048,1.1000,0.155242,0.2787,0.024194,2.508064,0.018145,1.471766,0.102823,0.08629,1619.433198
8,KSS,27.67,722,-21.28,0.034274,0.8167,0.118952,0.1747,0.010081,9.163331,0.225806,1.245997,0.068548,0.091532,722.804481
9,SYF,37.94,527,7.03,0.108871,1.2000,0.178427,0.7345,0.106855,4.426202,0.044355,0.980498,0.044355,0.096573,527.148129


In [170]:
rv_dataframe.iloc[:,:10]
writer = pd.ExcelWriter('Value_Strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index= False)

PermissionError: [Errno 13] Permission denied: 'Value_Strategy.xlsx'

In [165]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00 ',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [166]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }
for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [167]:
writer.close()